# Notebook Describing Z Stack Linking Functionalities
This is very much a work in progress. Here, we describe our initial attempts at 3D visualization of a stack of images.
This will be a future direction.

We begin by importing functions such as NumPy, Pandas and skimage, as well as our functions from the modules `preprocess`, `properties` and `zstack`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage.measure import regionprops, regionprops_table
import pandas as pd

import cells_in_gel.preprocess as prep
import cells_in_gel.properties as props
from cells_in_gel.properties import im_properties
import cells_in_gel.zstack as z

We import our image stack from a file saved in the same folder. Later, this will be optimized to do in much higher throughput as our other examples, however, 3D visualizing one image takes a lot of time.

In [ ]:
im_stack = io.imread('C3-NTG-CFbs_NTG5ECM_1mMRGD_20x_003.tif')

We then create our label array that will be filled in with our `phalloidin_labeled` function, which segments and labels each cell in each z slice of the z stack.

In [ ]:
x, y, z = im_stack.shape # dimensions of the image stack
label_stack = np.zeros_like(im_stack) # create empty array for label

In [ ]:
for i in range(x):
    label_stack[i] = prep.phalloidin_labeled(im_stack[i])

We can see that as we scroll through the output of the `phalloidin_labeled`, that the colors don't always match up between cells,  i.e. in each slice it is labeling the same cell differently.

If we make a data frame of the labels using `im_properties`, we can see that there are over 1000 entries, as each cell, even as it progresses through the z stack,  is being counted as a separate entity.

In [ ]:
frames = []

#Create data frame called frames that has the image number labelled.
for i in range(len(label_stack)):
    frames.append(props.im_properties(label_stack[i], im_stack[i]))
    frames[i]['Image number'] = i

    #concatenate all the data into one big data frame
df = pd.concat(frames)

We call upon the `zlink` function and it identifies which cells are the same across the vertical slices and groups them together as one cell.

In [ ]:
dfnew = z.zlink(df)

In [ ]:
dfnew.head()

If we call `dfnew['cell'].max()` we see that it = 156, meaning there are 156 individual cells in this z stack.

Since we now have the center points (centroid 0 and centroid 1), we wanted to use these as seed values in a script to region grow those points into a full volume as the algorithm progresses through a z stack. To do this, we wanted to follow [this method](http://notmatthancock.github.io/2017/10/09/region-growing-wrapping-c.html). We needed to install and import mayavi and mlab. Additionally other things had to be installed to ensure this could run in jupyter notebooks, as detailed [here](https://docs.enthought.com/mayavi/mayavi/installation.html).

In [ ]:
import mayavi
from mayavi import mlab
mlab.init_notebook()

As a first proof of concept, we tried to plot the centroids of the cells as points, which worked:

In [ ]:
mlab.clf()
mlab.points3d(dfnew['centroid-0'], dfnew['centroid-1'], dfnew['Image number'], scale_factor = 20)

As a next proof of concept, we made a massive dataframe with all the coordinates from all the im_properties of every z slice, but this seemed to crash mayavi. So.. Still a work a progress.

In [ ]:
new_df = np.vstack(dfnew['coords'])

In [ ]:
lens = [len(item) for item in dfnew['coords']]
len(lens)
df_out = pd.DataFrame({"Image number" : np.repeat(dfnew['Image number'].values,lens), 
               "coords x" : list(zip(*new_df))[0],
                      "coords y":list(zip(*new_df))[1]})


In [ ]:
mlab.clf()
mlab.points3d(df_out['coords x'], df_out['coords y'], df_out['Image number'], scale_factor = 20)